In [1]:
import sys
import os
from selenium import webdriver
import json
from PIL import Image
import pytesseract
import time
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from ReportMonitor_UI import Ui_MainWindow
import threading
from win10toast import ToastNotifier
from Notification import send_notification
from collections import defaultdict
import winsound
import string
import importlib

In [2]:
# driver.get("http://" + goalurl + "/pyxx/login.aspx")
# driver.maximize_window()
# current_window_size = driver.get_window_size()
# width = current_window_size["width"]
# height = current_window_size["height"]
# driver.set_window_size(width, height)
# aElement = driver.find_element_by_xpath('./*//title')
# print(aElement.get_attribute("innerText"))

In [3]:
user = "2017400144"
goalurl = "yjsy.buct.edu.cn:8080"
url = "http://" + goalurl + "/pyxx/txhdgl/hdlist.aspx?xh=" + user

In [6]:
option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
option.add_argument('window-size=1536, 824')
option.add_argument('headless')

driver = webdriver.Chrome(options=option)
driver.get(url)
target = driver.find_element_by_xpath("./*//input[@name='txtyzm']/../img")
driver.execute_script("arguments[0].scrollIntoView(false);", target)  # 拖动到可见的元素去

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./*//input[@name='txtyzm']/../img"}
  (Session info: headless chrome=90.0.4430.72)


In [ ]:

yzmElement = driver.find_element_by_xpath("./*//input[@name='txtyzm']/../img")
img = Image.open('screenshot.png')

left = int(yzmElement.location['x'])

# top = int(yzmElement.location['y'])
# top = img.height - 32 - yzmElement.size['height'] #- bottomTable.size['height']
if int(yzmElement.location['y'] + yzmElement.size['height']) > img.height:
    bottom = img.height
else:
    bottom = int(yzmElement.location['y'] + yzmElement.size['height'])
top = bottom - yzmElement.size['height']
right = int(yzmElement.location['x'] + yzmElement.size['width'])
# bottom = int(yzmElement.location['y'] + yzmElement.size['height'])
print((left, top, right, bottom))
img = img.crop((left, top, right, bottom))
img.save('code.png')

In [7]:
def _get_threshold(image):
    pixel_dict = defaultdict(int)

    #  a dictionary of pixels and the number of occurrences of that pixel
    rows, cols = image.size
    for i in range(rows):
        for j in range(cols):
            pixel = image.getpixel((i, j))
            pixel_dict[pixel] += 1

    count_max = max(pixel_dict.values())  # gets the number of times a pixel appears
    pixel_dict_reverse = {v: k for k, v in pixel_dict.items()}
    threshold = pixel_dict_reverse[count_max]  #

    return threshold

def _get_bin_table(threshold):
    table = []
    for i in range(256):
        rate = 0.1  # threshold
        if threshold * (1 - rate) <= i <= threshold * (1 + rate):
            table.append(1)
        else:
            table.append(0)
    return table

def _cut_noise(image):
    rows, cols = image.size  #
    change_pos = []  #
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            pixel_set = []
            for m in range(i - 1, i + 2):
                for n in range(j - 1, j + 2):
                    if image.getpixel((m, n)) != 1:  # 1,0
                        pixel_set.append(image.getpixel((m, n)))
            if len(pixel_set) <= 4:
                change_pos.append((i, j))
    for pos in change_pos:
        image.putpixel(pos, 1)
    return image

In [19]:
image = Image.open('code.png')
image = image.convert('L')
max_pixel = _get_threshold(image)
table = _get_bin_table(threshold=max_pixel)
image = image.point(table, '1')
image = _cut_noise(image)
image.save('code_denoise.png')
yzm = pytesseract.image_to_string(image)
# exclude_char_list = ' .·:`‘、“\\|\'\"?![],()~@#$%^&*_+-={};<>/¥'
# yzm = ''.join([x for x in yzm if x not in exclude_char_list]).strip()

print(yzm)

whitespace = ['\f', '\n', '\r', '\t', '\v', '\u00A0', '\u2028', '\u2029']
yzm = ''.join(filter(lambda x: x not in whitespace, filter(lambda x: x in string.printable, yzm))).strip(' \r\n\f\t\b\v\0')


print(yzm)

 

 

0966

 

 

0966
